In [1]:
from inclearn.lib.CityscapesClassification import CityscapesClassification
import os
from torchvision import transforms, datasets

In [3]:
root='/home/romilb/datasets/cityscapes_raw'
min_res=1
split="train"
cities=next(os.walk(os.path.join(root, 'leftImg8bit', split)))[1]

In [5]:
",".join(cities)

'zurich,stuttgart,darmstadt,dusseldorf,monchengladbach,aachen,tubingen,bochum,bremen,cologne,ulm,jena,strasbourg,hamburg,krefeld,weimar,hanover,erfurt'

In [3]:
for city in cities:
    CityscapesClassification.generate_sample_list(root, splits=[split], subset_cities=[city], min_res=min_res, write_filename=str(city))

Creating train dataset on only ['leverkusen'] cities
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego 

ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rect

ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rectification border
rectification border
rectification border
out of roi
ego vehicle
rectification border
rect

ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle
out of roi
ego vehicle

# Test

In [97]:
c=CityscapesClassification(root, os.path.join(root, 'sample_lists', 'citywise', 'aachen'), transform=transforms.ToTensor())

Using data cache: False


In [98]:
import torchvision.models as models
import torch.nn as nn
import torch
import time
import numpy as np
import torch.optim as optim
import os
import copy

In [108]:
batch_size=128
dataloaders_dict = {x: torch.utils.data.DataLoader(c, batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

In [109]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [110]:
model = models.resnet18()

In [111]:
set_parameter_requires_grad(model, feature_extracting=True)

In [112]:
num_classes = len(np.unique(c.y))
print(num_classes)
model.fc = nn.Linear(512, num_classes)

6


In [113]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 fc.weight
	 fc.bias


In [114]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [121]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, dataloaders, criterion, optimizer, num_epochs=2):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                print(
                'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.
                format(epoch, batch_idx * len(inputs), len(dataloaders[phase]) * len(inputs),
                       100. * batch_idx / len(dataloaders[phase]), loss))

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [124]:
model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=3)

Epoch 0/2
----------
Train Epoch: 0 [0/12 (0%)]	Loss: 0.886654
Train Epoch: 0 [128/12 (8%)]	Loss: 0.754466
Train Epoch: 0 [256/12 (17%)]	Loss: 0.673177
Train Epoch: 0 [384/12 (25%)]	Loss: 0.827275
Train Epoch: 0 [512/12 (33%)]	Loss: 0.665579
Train Epoch: 0 [640/12 (42%)]	Loss: 0.658259
Train Epoch: 0 [768/12 (50%)]	Loss: 0.753187
Train Epoch: 0 [896/12 (58%)]	Loss: 0.639148
Train Epoch: 0 [1024/12 (67%)]	Loss: 0.783357
Train Epoch: 0 [1152/12 (75%)]	Loss: 0.804506
Train Epoch: 0 [1280/12 (83%)]	Loss: 0.897111
Train Epoch: 0 [616/12 (92%)]	Loss: 0.570007
train Loss: 0.7512 Acc: 0.7480
Train Epoch: 0 [0/12 (0%)]	Loss: 1.208531
Train Epoch: 0 [128/12 (8%)]	Loss: 1.067443
Train Epoch: 0 [256/12 (17%)]	Loss: 1.185200
Train Epoch: 0 [384/12 (25%)]	Loss: 1.187729
Train Epoch: 0 [512/12 (33%)]	Loss: 1.073853
Train Epoch: 0 [640/12 (42%)]	Loss: 1.214042
Train Epoch: 0 [768/12 (50%)]	Loss: 1.215423
Train Epoch: 0 [896/12 (58%)]	Loss: 1.173489
Train Epoch: 0 [1024/12 (67%)]	Loss: 1.187447
Train E

In [15]:
from ekya.models.resnet import Resnet
import torch
import pandas as pd

In [5]:
model = Resnet(6)

Params to learn:
	 fc.weight
	 fc.bias


In [40]:
trsf = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
sample_list_root = os.path.join(root, 'sample_lists', 'citywise')
sample_names = ['aachen', 'zurich']
c=CityscapesClassification(root, sample_names, sample_list_root, transform=trsf, resize_res=224)

Using data cache: False
TODO: Update the idxs!


In [41]:
c.samples["idx"] = pd.Series(range(0, len(c.samples["idx"]))).values
c.samples.set_index("idx", inplace=True)

In [42]:
print(c.samples)

                                                imgpath  class    x0   y0  \
idx                                                                         
0     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1166  399   
1     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1294  409   
2     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1329  371   
3     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1401  379   
4     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1567  378   
5     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1610  364   
6     train/aachen/aachen_000102_000019_leftImg8bit.png      1  1789  361   
7     train/aachen/aachen_000106_000019_leftImg8bit.png      1  1558  427   
8     train/aachen/aachen_000106_000019_leftImg8bit.png      1  1598  416   
9     train/aachen/aachen_000106_000019_leftImg8bit.png      1   206  391   
10    train/aachen/aachen_000106_000019_leftImg8bit.png      3   510  223   

In [26]:
batch_size=128
dataloaders_dict = {x: torch.utils.data.DataLoader(c, batch_size=batch_size, shuffle=True, num_workers=0) for x in ['train', 'val']}

In [27]:
model.train_model(dataloaders_dict)

Epoch 0/0
----------
Train Epoch: 0 [0/1536 (0%)]	Loss: 0.817928
Train Epoch: 0 [128/1536 (8%)]	Loss: 0.970138
Train Epoch: 0 [256/1536 (17%)]	Loss: 1.170425
Train Epoch: 0 [384/1536 (25%)]	Loss: 1.222084
Train Epoch: 0 [512/1536 (33%)]	Loss: 0.701257
Train Epoch: 0 [640/1536 (42%)]	Loss: 0.893914
Train Epoch: 0 [768/1536 (50%)]	Loss: 0.945100
Train Epoch: 0 [896/1536 (58%)]	Loss: 0.968932
Train Epoch: 0 [1024/1536 (67%)]	Loss: 0.763524
Train Epoch: 0 [1152/1536 (75%)]	Loss: 0.800863
Train Epoch: 0 [1280/1536 (83%)]	Loss: 0.846183
Train Epoch: 0 [616/672 (92%)]	Loss: 0.675793
train phase done. Loss: 0.9089 Acc: 0.7295
Train Epoch: 0 [0/1536 (0%)]	Loss: 0.880125
Train Epoch: 0 [128/1536 (8%)]	Loss: 0.762410
Train Epoch: 0 [256/1536 (17%)]	Loss: 0.881020
Train Epoch: 0 [384/1536 (25%)]	Loss: 0.875434
Train Epoch: 0 [512/1536 (33%)]	Loss: 0.846387
Train Epoch: 0 [640/1536 (42%)]	Loss: 0.885644
Train Epoch: 0 [768/1536 (50%)]	Loss: 0.846897
Train Epoch: 0 [896/1536 (58%)]	Loss: 0.912507
Tr

(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU